# Bokeh로 나타내보는 전문가 평점과 네티즌 평점관의 관계 

In [1]:
from bokeh.models import HoverTool
from scipy import stats
from statsmodels.formula.api   import  ols

from bokeh.plotting import output_notebook, figure, show
import os
import pandas as pd

output_notebook()

df = pd.read_csv('./movie.csv',encoding='CP949')

group_by_netizen = df.groupby('grade')['netizen_score'].agg(['mean','std'])
group_by_expert = df.groupby('grade')['expert_score'].agg(['mean','std'])   ## std 표준편차 

print(group_by_netizen)
print(group_by_expert)

see_12_frame  = df.loc[df.grade == '12세 관람가']
see_15_frame  = df.loc[df.grade == '15세 관람가']
see_adult_frame  = df.loc[df.grade == '청소년 관람불가']
see_all_frame = df.loc[df.grade == '전체 관람가']

hover = HoverTool(
    tooltips=[
        ("index", "$index"),
        ("movie_id", "@movie_id"),
        ("movie_name", "@movie_name"),
        ("netizen_score", "@netizen_score"),  
        ("expert_score", "@expert_score"),
        ("grade", "@grade"),
        ("genre", "@genre")        
    ]
)
p = figure(tools=[hover],plot_width=500, plot_height=300)

p.scatter("expert_score", "netizen_score", source = df)

show(p)

Loading BokehJS ...

              mean       std
grade                       
12세 관람가   8.636897  0.830633
15세 관람가   8.337273  0.796159
전체 관람가    8.932308  0.637536
청소년 관람불가  7.536875  1.163734
              mean       std
grade                       
12세 관람가   6.621724  0.976678
15세 관람가   6.637273  1.144428
전체 관람가    6.881538  0.893251
청소년 관람불가  5.681875  2.387708


# 관람가에 따로 나누어 보기 쉽게 색변환

In [2]:
from bokeh.transform import factor_cmap, factor_mark

temp_grade = ['12세 관람가', '15세 관람가', '전체 관람가', '청소년 관람불가']
colors = ['black', 'orange', 'green', 'red']

p = figure(plot_width=500, plot_height=300)

p.scatter(x="expert_score", y="netizen_score",alpha=0.5, source=df,
    color=factor_cmap('grade', colors, temp_grade), legend='grade')

show(p)



# 12세 관람가

In [3]:
from bokeh.models import Title

p = figure(plot_width=500, plot_height=300)

p.scatter(x="expert_score", y="netizen_score",alpha=0.5, source=see_12_frame,
    color='black')
p.title = Title(text='12세 관람가의 평점(x축이 전문가 평점,y축이 네티즌 평점)')
show(p)

# 15세 관람가

In [4]:
p = figure(plot_width=500, plot_height=300)

p.scatter(x="expert_score", y="netizen_score",alpha=0.5, source=see_15_frame,
    color='orange')
p.title = Title(text='15세 관람가의 평점(x축이 전문가 평점,y축이 네티즌 평점)')
show(p)

# 전체관람가

In [5]:
p = figure(plot_width=500, plot_height=300)

p.scatter(x="expert_score", y="netizen_score",alpha=0.5, source=see_all_frame,
    color='green')
p.title = Title(text='전체 관람가의 평점(x축이 전문가 평점,y축이 네티즌 평점)')
show(p)

# 청소년 관람불가 

In [6]:
p = figure(plot_width=500, plot_height=300)

p.scatter(x="expert_score", y="netizen_score",alpha=0.5, source=see_adult_frame,
    color='red')
p.title = Title(text='청소년 관람불가의 평점(x축이 전문가 평점,y축이 네티즌 평점)')
show(p)

# 관람등급로 예측해보는 네티즌 평점

In [7]:
Rformula = 'netizen_score ~ expert_score + grade'

regression_result = ols(Rformula, data = df).fit()
##print(regression_result)

sample1 = df[df.columns.difference(['netizen_score'])][0:][:]
##print(sample1)

sample1_predict = pd.DataFrame(regression_result.predict(sample1))
sample1_predict.columns = ['predict_netizen_score']
##print(sample1_predict)

sample1_results = pd.concat([sample1_predict,sample1], axis=1)

##print(sample1_results)

hover = HoverTool(
    tooltips=[
        ("index", "$index"),
        ("movie_id", "@movie_id"),
        ("movie_name", "@movie_name"),
        ("netizen_score", "@netizen_score"),  
        ("predict_netizen_score", "@predict_netizen_score"),
        ("grade", "@grade"),
        ("genre", "@genre")        
    ]
)

p = figure(tools=[hover],plot_width=500, plot_height=300)

p.scatter("expert_score", "predict_netizen_score", source = sample1_results, color = 'red')

show(p)